In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
limit = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\soham\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import os

print(os.path.isdir("tmdb_5000_credits"))  # True if it's a directory
print(os.path.isfile("tmdb_5000_credits"))  # True if it's a file

False
False


In [4]:
file_path1 = r"C:\Users\soham\OneDrive\Desktop\PY\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_credits.csv"
data1 = pd.read_csv(file_path1)
file_path2 = r"C:\Users\soham\OneDrive\Desktop\PY\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_movies.csv"
data2 = pd.read_csv(file_path2)

In [5]:
data1 = data1[['movie_id', 'cast', 'crew']]


In [6]:
data2.rename(columns={'id': 'movie_id'}, inplace=True)


In [7]:
movies_data = pd.merge(data1,data2,on = "movie_id",how = "inner")

In [8]:
movies = movies_data.drop(['original_language', 'original_title','popularity','revenue','runtime','status','vote_average','vote_count','budget','homepage','spoken_languages','tagline'	],axis=1)

In [9]:
movies.isnull().sum()

movie_id                0
cast                    0
crew                    0
genres                  0
keywords                0
overview                3
production_companies    0
production_countries    0
release_date            1
title                   0
dtype: int64

In [10]:
movies.dropna(inplace = True)

In [11]:
def change (ndarray,string ):
    genre = []
    for item in ast.literal_eval(ndarray):
      genre.append(item[string])

    return genre        
     

In [12]:
movies['genres'] =  movies['genres'].apply(lambda n:change(n,'name'))
movies['cast'] =  movies['cast'].apply(lambda n:change(n,'name')).apply(lambda x:x[:1])
movies['keywords'] =  movies['keywords'].apply(lambda n:change(n,'name'))
movies['production_companies'] =  movies['production_companies'].apply(lambda n:change(n,'name'))
movies['production_countries'] =  movies['production_countries'].apply(lambda n:change(n,'name'))
movies['crew'] = movies['crew'].apply(lambda n: [d['name'] for d in ast.literal_eval(n) if d['department'] == 'Directing'])

In [13]:
movies.head()

,movie_id,cast,crew,genres,keywords,overview,production_companies,production_countries,release_date,title
0,19995,[Sam Worthington],[James Cameron],"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,Avatar
1,285,[Johnny Depp],"[Gore Verbinski, Pamela Alch, Sharron Reynolds...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,Pirates of the Caribbean: At World's End
2,206647,[Daniel Craig],"[Sam Mendes, Jayne-Ann Tenggren, Nicoletta Man...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,Spectre
3,49026,[Christian Bale],"[Christopher Nolan, Steve Gehrke, Matthew Reed...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,The Dark Knight Rises
4,49529,[Taylor Kitsch],"[Andrew Stanton, Annie Penn, Tommy Gormley, Be...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",[Walt Disney Pictures],[United States of America],2012-03-07,John Carter


In [14]:
movies['merge1'] = movies['genres'] + movies['cast'] + movies['keywords'] + movies['production_companies'] + movies['production_countries'] + movies['crew']

In [15]:
movies['merge1'] = movies['merge1'].apply(lambda ndarr: " ".join(st.replace(" " ,"") for st in ndarr))

In [16]:
movies['tags'] = movies['merge1'] + movies['overview']

In [17]:
movies = movies[['movie_id','release_date','title','tags']]

In [18]:
movies['tags'] = movies['tags'].apply(lambda n:" ".join([limit.lemmatize(w) for w in word_tokenize(n)]))

In [19]:
total_words = movies['tags'].apply(lambda x: len(word_tokenize(str(x)))).sum()
total_words

np.int64(361375)

In [20]:
CountVectorizer().get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [21]:
Vectorizer = CountVectorizer(max_features = 10800,stop_words = 'english',strip_accents ='ascii')

In [22]:
vect = Vectorizer.fit_transform(movies['tags']).toarray()

In [23]:
print(Vectorizer.get_feature_names_out())

['000' '007' '10' ... 'zoo' 'zooey' 'zookeeper']


In [24]:
cosine_vect = cosine_similarity(vect)

In [25]:
def find (movie):
    index = movies.index[movies['title'] == movie]
    c_v = cosine_vect[index][0]
    c_v[index] = 0
    ind = []
    for i in np.arange(10):
      pos = np.argmax(c_v)
      c_v[pos] = 0
      ind.append(pos)
        
    return titles(ind)
   

In [26]:
def titles (lst):
    df = pd.DataFrame(columns =['title','release_date'])
    for i in lst:
        row = movies.loc[[i],['title','release_date']]
        df =pd.concat([df, row],axis = 0,ignore_index = True)

    print(df)    

In [27]:
movies.to_csv('movie.csv')

In [28]:
c_v_df =pd.DataFrame(cosine_vect)

In [29]:
c_v_df.to_csv('cosine_vect',index = False)